In [1]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

#Progres bar
from tqdm import tqdm

In [2]:
#Load output.csv into df
with open('output.csv', encoding='UTF-8', mode='r') as input_file:
    df = pd.read_csv(input_file)

In [3]:
# Prep data
df[['sender']] = df[['sender']].fillna('Deleted Account')
users = df['sender'].unique()
msg_df = df.set_index('msg_id')

In [4]:
from functools import partial
# Create a new column with the sender of what each message is replying to
msg_df['reply_to_name'] = msg_df[msg_df['reply_to_msg_id'] != -1]['reply_to_msg_id'] \
    .map(partial(msg_df['sender'].get, default=np.NaN))
# Count the amount of times each replier-sender pair occurs
commonPairings = msg_df[msg_df['reply_to_name'].notnull()] \
    .reset_index() \
    .set_index(['sender', 'reply_to_name', 'msg_id'])['reply_to_msg_id'] \
    .groupby(['sender', 'reply_to_name'], ) \
    .count() \
    .sort_values(ascending=False)
# Get top 10, format, and write to csv
reduced = commonPairings.reset_index()[:10]
reduced['Pair'] = reduced['sender'] + ' -> ' + reduced['reply_to_name']
reduced = reduced.drop(['sender', 'reply_to_name'], axis=1) \
    .rename(columns={'reply_to_msg_id': 'Messages'}, )
reduced.to_csv("csvFiles/CommonPairings.csv", index=False)

In [5]:

from datetime import date

#Create a line graph showing activity of users each day
#Create new df with columns "all" then each user
#Add day column to df with just the first 10 characters of df['date']
df['day'] = df['date'].apply(lambda x: x[:10])

dailyActivity = df[['sender', 'day']] \
    .rename(columns={'day': 'Date'}) \
    .groupby(['Date']) \
    .value_counts() \
    .unstack(level=1) \
    .fillna(0) \
    .astype(int)
dailyActivity['all'] = dailyActivity.sum(axis=1)
dailyActivity = dailyActivity[['all'] + list(users)]

# dailyActivity2 = pd.DataFrame(columns=['all'] + list(users))
# dailyActivity3 = pd.DataFrame(columns=['all'] + list(users))
# pbar = tqdm(total=len(df['day'].unique()))
# for i, specificDate in enumerate(df['day'].unique()):
#     pbar.update(1)
#     #Get all messages sent on this date
#     specificDateMessages = df[df['day'] == specificDate]
#     dailyActivity2.loc[specificDate] = specificDateMessages['sender'].value_counts().fillna(0)
#     dailyActivity2.loc[specificDate]['all'] = len(specificDateMessages)
    
#     # #Add an empty row to dailyActivity for this date, with the row label being the date
#     # dailyActivity3.loc[specificDate] = [0] * (len(users) + 1)
#     # #The first column is the number of messages sent on this date
#     # dailyActivity3.loc[specificDate, 'all'] = len(specificDateMessages)
#     # #For each message sent on this date, add 1 to the corresponding user's row
#     # for k, row in specificDateMessages.iterrows():
#     #     dailyActivity3.loc[specificDate, row['sender']] += 1
# #Replace every NaN with 0 and convert everything to int
# dailyActivity2 = dailyActivity2.fillna(0).astype(int)
# pbar.close()

#Group the rows of dailyActivity by month
#If the first 7 characters of the row name are the same, then they are in the same month
#Add up the columns for all rows in the same month
dailyActivityByMonth = dailyActivity.groupby(lambda x: x[:7]).sum()

#Label index
dailyActivity.index.name = 'Date'
dailyActivityByMonth.index.name = 'Month'

dailyActivity.to_csv("csvFiles/DailyActivity.csv")
dailyActivityByMonth.to_csv("csvFiles/DailyActivityByMonth.csv")

In [6]:
#Calculate hourly activity
#Create new df with columns "all" then each user
hourlyActivity = pd.DataFrame(columns=['all'] + list(users))

#Add new column "time" and "hour" to df 11:19
df['time'] = df['date'].apply(lambda x: x[11:19])
df['hour'] = df['time'].apply(lambda x: int(x[:2]))

dfHourGrouped = df[['sender', 'hour']] \
    .groupby(['hour', 'sender']) \
    .value_counts() \
    .unstack(level=1) \
    .fillna(0) \
    .astype(int)
dfHourGrouped['all'] = dfHourGrouped.sum(axis=1)
dfHourGrouped = dfHourGrouped[['all'] + list(users)]

dfHourGrouped.index.name = 'Hour'
dfHourGrouped.to_csv("csvFiles/HourlyActivity.csv")

In [7]:
from emoji import UNICODE_EMOJI
def is_emoji(s):
    return s in UNICODE_EMOJI['en']

#Add a column with the number of characters in the message
df['messageLength'] = df['msg_content'].apply(lambda x: len(str(x)))
#Add a column with the number of capital characters in each message
df['capitalLetters'] = df['msg_content'].apply(lambda x: sum(1 for c in str(x) if c.isupper()))
#Add a column with the number of emojis in each message
df['emojiCount'] = df['msg_content'].apply(lambda x: sum(1 for c in str(x) if is_emoji(c)))
#Add a column with 1 if message is all caps, 0 otherwise
df['allCaps'] = df['msg_content'].apply(lambda x: 1 if str(x).isupper() else 0)

#Make a new dataframe where each row is a user
userData = pd.DataFrame()
#Add column for num messages sent
for row in users:
    #Add column for num messages sent
    userData.loc[row, 'numMessages'] = len(df[df['sender'] == row])
    #Add column for percentage for num emojis sent
    userData.loc[row, 'emojiPercentage'] = (df[df['sender'] == row]['emojiCount'].sum() / df[df['sender'] == row]['messageLength'].sum()) * 100
    #Add column for percentage for num capital letters sent
    userData.loc[row, 'capitalLettersPercentage'] = (df[df['sender'] == row]['capitalLetters'].sum() / df[df['sender'] == row]['messageLength'].sum()) * 100
    #Add column for percentage of all caps messages
    userData.loc[row, 'allCapsPercentage'] = (df[df['sender'] == row]['allCaps'].sum() / userData['numMessages'][row]) * 100
userData.index.name = 'User'
userData.to_csv("csvFiles/UserData.csv")
